In [140]:
from pathlib import Path
import pandas as pd

DATA_DIR = Path('DATA')
dfile = DATA_DIR/'Tuesday-WorkingHours.pcap_ISCX.csv'

pd.concat((pd.read_csv(DATA_DIR/f) for f in ['Tuesday-WorkingHours.pcap_ISCX.csv', 'Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv' ]), ignore_index=True)
#dfile = DATA_DIR / '-WorkingHours.pcap_ISCX.csv'
df = pd.read_csv(dfile, index_col=False)
df = df.rename(columns=lambda x: x.strip())

In [141]:
QUANT_COLS =  ['Flow Duration',
       'Total Fwd Packets', 'Total Backward Packets',
       'Total Length of Fwd Packets', 'Total Length of Bwd Packets',
       'Fwd Packet Length Max', 'Fwd Packet Length Min',
       'Fwd Packet Length Mean', 'Fwd Packet Length Std',
       'Bwd Packet Length Max', 'Bwd Packet Length Min',
       'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s',
       'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max',
       'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std',
       'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean',
       'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags',
       'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length',
       'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s',
       'Min Packet Length', 'Max Packet Length', 'Packet Length Mean',
       'Packet Length Std', 'Packet Length Variance', 'FIN Flag Count',   'SYN Flag Count', 'RST Flag Count', 'PSH Flag Count', 'ACK Flag Count',
       'URG Flag Count', 'CWE Flag Count', 'ECE Flag Count', 'Down/Up Ratio',
       'Average Packet Size', 'Avg Fwd Segment Size', 'Avg Bwd Segment Size',
       'Fwd Header Length.1', 'Fwd Avg Bytes/Bulk', 'Fwd Avg Packets/Bulk',
       'Fwd Avg Bulk Rate', 'Bwd Avg Bytes/Bulk', 'Bwd Avg Packets/Bulk',
       'Bwd Avg Bulk Rate', 'Subflow Fwd Packets', 'Subflow Fwd Bytes',
       'Subflow Bwd Packets', 'Subflow Bwd Bytes', 'Init_Win_bytes_forward',
       'Init_Win_bytes_backward', 'act_data_pkt_fwd', 'min_seg_size_forward','Active Mean', 'Active Std', 'Active Max', 'Active Min', 'Idle Mean',
       'Idle Std', 'Idle Max', 'Idle Min']

In [132]:
from sklearn.model_selection import train_test_split
import numpy as np

#df = df[(df['Label'] == 'FTP-Patator')]
df = df[QUANT_COLS + ['Label']]

df = df[~df.isin([np.nan, np.inf, -np.inf]).any(1)]
X_train, X_test, y_train, y_test = train_test_split(df.drop(["Label"], axis=1), df["Label"], test_size=1 / 3)

/tmp/ipykernel_16194/121367529.py:7: FutureWarning: In a future version of pandas all arguments of DataFrame.any and Series.any will be keyword-only.
  df = df[~df.isin([np.nan, np.inf, -np.inf]).any(1)]


In [116]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
sel = SelectFromModel(RandomForestClassifier(n_estimators = 200))
sel.fit(X_train, y_train)


SelectFromModel(estimator=RandomForestClassifier(n_estimators=200))

In [117]:
np.array(QUANT_COLS[:-1])[np.array(sel.get_support())]

array(['Total Backward Packets', 'Total Length of Fwd Packets',
       'Total Length of Bwd Packets', 'Fwd Packet Length Max',
       'Fwd Packet Length Mean', 'Fwd Packet Length Std',
       'Bwd Packet Length Max', 'Bwd Packet Length Mean',
       'Bwd Packet Length Std', 'Flow IAT Max', 'Fwd PSH Flags',
       'Fwd Header Length', 'Bwd Header Length', 'Max Packet Length',
       'Packet Length Mean', 'Packet Length Std',
       'Packet Length Variance', 'SYN Flag Count', 'Average Packet Size',
       'Avg Fwd Segment Size', 'Avg Bwd Segment Size',
       'Subflow Fwd Bytes', 'Subflow Bwd Packets', 'Subflow Bwd Bytes',
       'Init_Win_bytes_forward', 'Init_Win_bytes_backward',
       'act_data_pkt_fwd'], dtype='<U27')

In [145]:
from cur import cur_decomposition
import numpy as np
df = df[QUANT_COLS]
M = df.to_numpy()
M = M[~np.isnan(M).any(axis=1), :]
M = M[~np.isinf(M).any(axis=1), :]

C, U, R, c_idx = cur_decomposition(M, 40)

In [147]:
np.unique(np.array(QUANT_COLS)[np.array(c_idx)])

array(['Bwd IAT Total', 'Flow Bytes/s', 'Flow Duration', 'Flow IAT Max',
       'Fwd Header Length', 'Fwd Header Length.1', 'Fwd IAT Max',
       'Fwd IAT Total', 'Idle Min'], dtype='<U27')

In [148]:
len(QUANT_COLS)

77